In [1]:
#Results were produced in stints. This is the number of the last stint.
run = 2

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from scipy import stats
import joblib

#folder for saving results
filepath = ".../Resultate_final/VarAnn/saved_sim/"

In [3]:
#parameters as in section 4.3 of 'Assessing Asset-Liability risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
#and section 5.1 of 'A Least-Squares Monte Carlo Approach to the Estimation of Enterprise Risk' (Ha, Bauer 2019)
#parameters for q
q_0 = 4.605
m = 0.05
sigma_S = 0.18
#parameters for r
r_0 = 0.025
zeta = 0.25
gamma = 0.02
sigma_r = 0.01
lambd = 0.02
gamma_bar = gamma - (lambd*sigma_r)/zeta
#parameters for mu_(55+t)
mu_55 = 0.01
kappa = 0.07
sigma_mu = 0.0012
#parameters for brownian motion
rho_12 = -0.3
rho_13 = 0.06
rho_23 = -0.04
cov_mat = np.array([[1,rho_12,rho_13],[rho_12,1,rho_23],[rho_13,rho_23,1]])
#horizon parameters
tau = 1
T = 15
b = 10.792
#risk measure parameters
alpha_VaR = 0.995
alpha_ES = 0.99
alpha_Glue = 0.95
beta_Glue = 0.995
omega_Glue = np.array([1/3,1/3])
#Sizes for training, validation, test, and set size for Monte Carlo estimation of the risk measures
M_1 = 1500000
M_2 = 500000
M_3 = 500000
#ignore N or N_2 in the following. Was kept just in case, but not used.
N_2 = 1
M_MC = 500000

In [4]:
#Definition of these functions analogously to the appendix of 'A Least-Squares Monte Carlo Approach to the Estimation of Enterprise Risk' (Ha, Bauer 2019)
def B_r(t,T):
    return ((1-np.exp(-zeta*(T-t)))/zeta)
def B_mu(t,T):
    return ((np.exp(kappa*(T-t))-1)/kappa)
def A(t,T):
    tmp1 = gamma_bar*(B_r(t,T)-(T-t))
    tmp2 = (sigma_r/zeta)**2 * ((T-t) - 2*B_r(t,T) + (1-np.exp(-2*zeta*(T-t)))/(2*zeta))
    tmp3 = (sigma_mu/kappa)**2 * ((T-t) - 2*B_mu(t,T) + (np.exp(2*kappa*(T-t))-1)/(2*kappa))
    tmp4 =  2*rho_23*sigma_r*sigma_mu/(zeta*kappa) * (B_mu(t,T) - (T-t) + B_r(t,T) - (1-np.exp(-(zeta-kappa)*(T-t)))/(zeta-kappa))
    return np.exp(tmp1+0.5*(tmp2+tmp3+tmp4))
#Definition of this function analogously to section 4.3 of 'Assessing Asset-Liability risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
def F(t,k,r_t,mu_xt):
    return (A(t,t+k)*np.exp(-B_r(t,t+k)*r_t - B_mu(t,t+k)*mu_xt))

#parameters of the normal distribution of X_tau according to the appendix of 'A Least-Squares Monte Carlo Approach to the Estimation of Enterprise Risk' (Ha, Bauer 2019)
mu_q_tau = q_0 + (m-0.5*(sigma_S**2))*tau
mu_r_tau = r_0*np.exp(-zeta*tau) + gamma*(1-np.exp(-zeta*tau))
mu_mu_55_tau = mu_55*np.exp(kappa*tau)
mean_tau = np.array([mu_q_tau, mu_r_tau, mu_mu_55_tau])

cov_q_r_tau = rho_12*sigma_S*sigma_r*B_r(0,tau)
cov_q_mu_tau = rho_13*sigma_S*sigma_mu*B_mu(0,tau)
cov_r_mu_tau = rho_23*sigma_r*sigma_mu* ((1-np.exp(-(zeta-kappa)*tau))/(zeta-kappa))
var_q_tau = (sigma_S**2) * tau
var_r_tau = (sigma_r**2) * ((1-np.exp(-2*zeta*tau))/(2*zeta))
var_mu_tau = (sigma_mu**2) * ((np.exp(2*kappa*tau)-1)/(2*kappa))
cov_mat_tau = np.array([[var_q_tau, cov_q_r_tau, cov_q_mu_tau], [cov_q_r_tau, var_r_tau, cov_r_mu_tau], [cov_q_mu_tau, cov_r_mu_tau, var_mu_tau]])
C_tau = np.linalg.cholesky(cov_mat_tau)

#variance/covariance parameters of the conditional normal distribution of X_T according to the appendix of 'A Least-Squares Monte Carlo Approach to the Estimation of Enterprise Risk' (Ha, Bauer 2019)
var_q_T_cond = (sigma_S**2) *(T-tau) + ((sigma_r/zeta)**2) * (T-tau - 2*(1-np.exp(-zeta*(T-tau)))/zeta + (1-np.exp(-2*zeta*(T-tau)))/(2*zeta)) + (2*rho_12*sigma_S*sigma_r/zeta) * (T-tau- (1-np.exp(-zeta*(T-tau)))/zeta)
cov_q_r_T_cond = rho_12*sigma_S*sigma_r*((1-np.exp(-zeta*(T-tau)))/zeta) + ((sigma_r**2)/zeta) * ((1-2*np.exp(-zeta*(T-tau))+np.exp(-2*zeta*(T-tau)))/(2*zeta))
cov_q_mu_T_cond = rho_13*sigma_S*sigma_mu *((np.exp(kappa*(T-tau))-1)/kappa) + (rho_23*sigma_r*sigma_mu/zeta) * ((np.exp(kappa*(T-tau))-1)/kappa - (1-np.exp(-(zeta-kappa)*(T-tau)))/(zeta-kappa))
var_r_T_cond = (sigma_r**2) * ((1-np.exp(-2*zeta*(T-tau)))/(2*zeta))
cov_r_mu_T_cond = rho_23*sigma_r*sigma_mu*((1-np.exp(-(zeta-kappa)*(T-tau)))/(zeta-kappa))
var_mu_T_cond = (sigma_mu**2) * ((np.exp(2*kappa*(T-tau))-1)/(2*kappa))
cov_mat_T_cond = np.array([[var_q_T_cond, cov_q_r_T_cond, cov_q_mu_T_cond], [cov_q_r_T_cond, var_r_T_cond, cov_r_mu_T_cond], [cov_q_mu_T_cond, cov_r_mu_T_cond, var_mu_T_cond]])

#function for generating simulated risk factors X_tau and corresponding payments Y
def data_gen(M,N,Z,Z_T):
    #simulation of X_tau
    X_pre = np.transpose(np.matmul(C_tau,np.transpose(Z))) + np.tile(mean_tau, (M,1))
    X_tau = np.tile(X_pre, (N,1))
    np.random.shuffle(X_tau)

    #simulation of X_T given X_tau
    mu_q_T_cond = X_tau[:,0] + B_r(tau,T)*X_tau[:,1] + (gamma_bar - (sigma_r/zeta)**2)*(T-tau - (1-np.exp(-zeta*(T-tau)))/zeta) + 0.5*((sigma_r/zeta)**2) * ((1-np.exp(-zeta*(T-tau)))/zeta - (np.exp(-zeta*(T-tau))-np.exp(-2*zeta*(T-tau)))/zeta) - ((rho_23*sigma_r*sigma_mu)/kappa) * ( (np.exp(kappa*(T-tau))-1)/(kappa*(zeta-kappa)) - (np.exp(kappa*(T-tau))-np.exp(-(zeta-kappa)*(T-tau)))/(zeta*(zeta-kappa)) - (1/zeta) * (T-tau - (1-np.exp(-zeta*(T-tau)))/zeta)) -0.5*(sigma_S**2) * (T-tau) - (rho_12*sigma_S*sigma_r/zeta) * (T-tau - (1-np.exp(-zeta*(T-tau)))/zeta) - (rho_13*sigma_S*sigma_mu/kappa) * ((np.exp(kappa*(T-tau))-1)/kappa -T+tau)
    mu_r_T_cond = np.exp(-zeta*(T-tau))*X_tau[:,1] + (gamma_bar-(sigma_r/zeta)**2)*(1-np.exp(-zeta*(T-tau))) + 0.5*((sigma_r/zeta)**2) *(1-np.exp(-2*zeta*(T-tau))) - (rho_23*sigma_r*sigma_mu/kappa) * ((1-np.exp(-(zeta-kappa)*(T-tau)))/(zeta-kappa) - (1-np.exp(-zeta*(T-tau)))/zeta)
    mu_mu_T_cond = np.exp(kappa*(T-tau))*X_tau[:,2] - (rho_23*sigma_r*sigma_mu/zeta) * ((np.exp(kappa*(T-tau))-1)/kappa - (1-np.exp(-(zeta-kappa)*(T-tau)))/(zeta-kappa)) - ((sigma_mu**2)/kappa) * ((np.exp(2*kappa*(T-tau))-1)/(2*kappa) - (np.exp(kappa*(T-tau))-1)/kappa)
    mean_T_cond = np.array([mu_q_T_cond, mu_r_T_cond, mu_mu_T_cond])

    X_T = np.empty((M*N,3))
    X_T = Z_T + np.transpose(mean_T_cond)

    #calculation of Y from X_T and X_tau
    Y = F(t=tau, k=T-tau, r_t=X_tau[:,1], mu_xt=X_tau[:,2]) * np.maximum(np.exp(X_T[:,0]), b*np.sum([F(t=T, k=i, r_t=X_T[:,1], mu_xt=X_T[:,2]) for i in range(1,51)], axis=0))
    return X_tau, Y

#function for estimation of GlueVaR
def GlueVaR(omega, L, alpha, beta):
    j_beta = int(len(L)*(1-beta))-1
    j_alpha = int(len(L)*(1-alpha))-1

    ES_beta = 1/(1-beta) * np.sum(L[0:j_beta-1])/len(L) + ( 1 - (j_beta-1)/((1-beta)*len(L)) )*L[j_beta]
    ES_alpha = 1/(1-alpha) * np.sum(L[0:j_alpha-1])/len(L) + ( 1 - (j_alpha-1)/((1-alpha)*len(L)) )*L[j_alpha]
    VaR_alpha = L[j_alpha]

    return omega[0]*ES_beta + omega[1]*ES_alpha + (1-omega[0]-omega[1])*VaR_alpha

In [5]:
for j in range(100):
    #simulation of multivariate standard normal random variables
    Z_train = np.random.multivariate_normal(mean=np.full(3,0),cov=np.identity(3),size=int(M_1/N_2))
    Z_train_T = np.random.multivariate_normal(mean=np.full(3,0),cov=cov_mat_T_cond,size=M_1)
    Z_val = np.random.multivariate_normal(mean=np.full(3,0),cov=np.identity(3),size=M_2)
    Z_val_T = np.random.multivariate_normal(mean=np.full(3,0),cov=cov_mat_T_cond,size=M_2)
    Z_test = np.random.multivariate_normal(mean=np.full(3,0),cov=np.identity(3),size=M_3)
    Z_test_T = np.random.multivariate_normal(mean=np.full(3,0),cov=cov_mat_T_cond,size=M_3)
    Z_MC = np.random.multivariate_normal(mean=np.full(3,0),cov=np.identity(3),size=M_MC)
    Z_MC_T = np.random.multivariate_normal(mean=np.full(3,0),cov=cov_mat_T_cond,size=M_MC)
    
    #Calculate the risk factor X_tau and the corresponding simulated payoffs Y
    X_tau_train, Y_train = data_gen(M=int(M_1/N_2),N=N_2,Z=Z_train,Z_T=Z_train_T)
    X_tau_val, Y_val = data_gen(M=M_2,N=1,Z=Z_val,Z_T=Z_val_T)
    X_tau_test, Y_test = data_gen(M=M_3,N=1,Z=Z_test,Z_T=Z_test_T)

    #define and compile neural network model, setup as in section 4.3 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
    bi = np.log(np.sum(Y_train)/M_1)
    model = tf.keras.models.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=(3,)),
        tf.keras.layers.Dense(4, activation='tanh'),
        tf.keras.layers.Dense(4, activation='tanh'),
        tf.keras.layers.Dense(1, activation='exponential', bias_initializer=tf.keras.initializers.Constant(value=bi))])
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])

    #train the model
    hist = model.fit(x=X_tau_train, y=Y_train, epochs=40, batch_size=10000, validation_data=(X_tau_val,Y_val),verbose=0)


    #definition of random forest regressor
    rfr = RandomForestRegressor(n_estimators=400, criterion='squared_error', max_features=2, min_samples_leaf=1250, bootstrap=True, verbose=0, warm_start=True, n_jobs=-1)
    #training
    rfr.fit(X=X_tau_train,y=Y_train)

    #Calculating the parts of the test set that fall into B_1 and B_2
    q_70 = stats.norm.ppf(0.7, loc=mu_q_tau, scale=var_q_tau)
    q_30 = stats.norm.ppf(0.3, loc=mu_q_tau, scale=var_q_tau)
    r_70 = stats.norm.ppf(0.7, loc=mu_r_tau, scale=var_r_tau)
    r_30 = stats.norm.ppf(0.3, loc=mu_r_tau, scale=var_r_tau)
    B_1 = np.apply_along_axis(np.all, axis=1, arr=np.column_stack( (X_tau_test[:,0] > q_70, X_tau_test[:,1] < r_30)) )
    B_2 = np.apply_along_axis(np.all, axis=1, arr=np.column_stack( (X_tau_test[:,0] < q_30, X_tau_test[:,1] > r_70)) )

    #computation of the metrics (a), (b), (c) with B_1 and (c) with B_2 for the neural network
    Y_pred_NN = model.predict(X_tau_test)[:,0]
    mse_train_NN = hist.history['mse'][-1]
    mse_val_NN = hist.history['val_mse'][-1]
    mc_tmp = Y_pred_NN - Y_test
    metric_a_NN = np.sum(mc_tmp)/len(Y_test)
    metric_b_NN = np.sum((mc_tmp)*Y_pred_NN)/len(Y_test)
    metric_c_B_1_NN = np.sum(mc_tmp[B_1])/len(Y_test)
    metric_c_B_2_NN = np.sum(mc_tmp[B_2])/len(Y_test)

    #computation of the metrics (a), (b), (c) with B_1 and (c) with B_2 and training/validation MSE for the random forest
    mse_train_RF = mean_squared_error(y_pred=rfr.predict(X_tau_train),y_true=Y_train)
    Y_pred_RF = rfr.predict(X_tau_test)
    mse_val_RF = mean_squared_error(y_pred=Y_pred_RF,y_true=Y_test)
    mc_tmp = Y_pred_RF - Y_test
    metric_a_RF = np.sum(mc_tmp)/len(Y_test)
    metric_b_RF = np.sum((mc_tmp)*Y_pred_RF)/len(Y_test)
    metric_c_B_1_RF = np.sum(mc_tmp[B_1])/len(Y_test)
    metric_c_B_2_RF = np.sum(mc_tmp[B_2])/len(Y_test)
    
    print(mse_val_NN,mse_val_RF)

    #computation of risk measure estimators
    #simulation of risk factors X_tau, Y_MC not used in the following
    X_tau_MC, Y_MC = data_gen(M=M_MC,N=1,Z=Z_MC,Z_T=Z_MC_T)

    #Prediction of payoffs
    Y_pred_MC_NN = model.predict(X_tau_MC)[:,0]
    Y_pred_MC_RF = rfr.predict(X_tau_MC)

    L_NN = np.sort(Y_pred_MC_NN)[::-1]
    L_RF = np.sort(Y_pred_MC_RF)[::-1]

    #Value-at-Risk
    j_VaR = int(M_MC*(1-alpha_VaR))-1
    VaR_hat_NN = L_NN[j_VaR]
    VaR_hat_RF = L_RF[j_VaR]

    #Expected Shortfall
    j_ES = int(M_MC*(1-alpha_ES))-1
    ES_hat_NN = 1/(1-alpha_ES) * np.sum(L_NN[0:j_ES-1])/M_MC + ( 1 - (j_ES-1)/((1-alpha_ES)*M_MC) )*L_NN[j_ES]
    ES_hat_RF = 1/(1-alpha_ES) * np.sum(L_RF[0:j_ES-1])/M_MC + ( 1 - (j_ES-1)/((1-alpha_ES)*M_MC) )*L_RF[j_ES]
    
    #GlueVaR
    GlueVaR_hat_NN = GlueVaR(omega=omega_Glue, L=L_NN, alpha=alpha_Glue, beta=beta_Glue)
    GlueVaR_hat_RF = GlueVaR(omega=omega_Glue, L=L_RF, alpha=alpha_Glue, beta=beta_Glue)
    print('Estimates NN:',VaR_hat_NN,ES_hat_NN,GlueVaR_hat_NN)
    print('Estimates RF:',VaR_hat_RF,ES_hat_RF,GlueVaR_hat_RF)

    #save results for further evaluation
    output = np.array([[mse_train_NN,mse_val_NN,metric_a_NN,metric_b_NN,metric_c_B_1_NN,metric_c_B_2_NN,VaR_hat_NN,ES_hat_NN,GlueVaR_hat_NN],
                      [mse_train_RF,mse_val_RF,metric_a_RF,metric_b_RF,metric_c_B_1_RF,metric_c_B_2_RF,VaR_hat_RF,ES_hat_RF,GlueVaR_hat_RF]])

    joblib.dump(output,filepath+'output_'+str(run)+'_'+str(j)+'.joblib')
    #prints just for checking while the notebook is running
    print(j)

2021-11-06 19:28:10.612435: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-06 19:28:12.813923: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30988 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:16:00.0, compute capability: 7.0
2021-11-06 19:28:12.815645: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30988 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3a:00.0, compute capability: 7.0
2021-11-06 19:28:12.817224: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:

7805.6220703125 2571.0603131204566
Estimates NN: 141.01164 142.70219797363282 134.4103130102539
Estimates RF: 140.14151027599502 140.9873833960955 133.07638012605685
0
5009.64111328125 2584.443958001435
Estimates NN: 135.3646 136.81617299194338 129.7357093475342
Estimates RF: 141.35616233922906 143.2179033063888 134.48067115117533
1
2667.051025390625 2935.7414431081106
Estimates NN: 138.75632 140.09485488891602 132.76808146362305
Estimates RF: 140.46937227139125 141.2871205403894 133.09151900604758
2
2584.733642578125 2576.4332171291953
Estimates NN: 140.02068 141.48110286254885 133.86639440897625
Estimates RF: 138.88780620159153 139.48276206252353 132.5583874927487
3
2579.8359375 2557.7361606373397
Estimates NN: 138.05824 139.27988595581056 132.02936001098632
Estimates RF: 140.49241034150947 141.10918359465978 133.18303566707348
4
2607.15234375 2576.6018451837717
Estimates NN: 142.52025 144.22657910766603 135.50077995707196
Estimates RF: 142.1415686010747 142.81348318402868 134.095800